In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
# import geopandas as gpd
import numpy as np
import os
import scipy as sc
import requests
import datetime
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from workalendar.europe import Italy

pal = sns.cubehelix_palette(8, rot=-.222, dark=.1)

In [50]:
w = pd.read_csv('../traffic/T.csv')
file = pd.read_csv('../traffic/Traffic_dataset_v0.1.csv', parse_dates=['DATETIME'], date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M'))
file.head()

,ID_SENSOR,DIRECTION,DATETIME,NUM_VEHICLES,SPEED_MEAN,SPEED_SD,EVENT,LON,LAT
0,1,A,2018-09-01 02:00:00,NaN,NaN,NaN,NO_EVENT,12.291754,41.800588
1,1,A,2018-09-01 02:15:00,NaN,NaN,NaN,NO_EVENT,12.291754,41.800588
2,1,A,2018-09-01 02:30:00,NaN,NaN,NaN,NO_EVENT,12.291754,41.800588
3,1,A,2018-09-01 02:45:00,NaN,NaN,NaN,NO_EVENT,12.291754,41.800588
4,1,A,2018-09-01 03:00:00,NaN,NaN,NaN,NO_EVENT,12.291754,41.800588


In [51]:
file_new = file.dropna(subset=['NUM_VEHICLES', 'SPEED_MEAN', 'SPEED_SD'], axis=0, thresh=2)
file_new = file_new.reset_index()

In [52]:
w.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1, inplace=True)

In [53]:
file_final = pd.concat([file_new, w], axis=1)

In [54]:
file_final.drop(['index', 'Precipitation'], axis=1, inplace=True)
file_final = file_final.dropna(subset=['SPEED_SD'], axis=0)
file_final = file_final.reset_index()

In [55]:
file_final.DIRECTION = pd.get_dummies(file_final.DIRECTION, prefix = 'A', drop_first=True)

In [56]:
file_final['prev15'] = file_final.NUM_VEHICLES.diff()
file_final['prev30'] = file_final.NUM_VEHICLES.diff(periods=2)
file_final.head()

,index,ID_SENSOR,DIRECTION,DATETIME,NUM_VEHICLES,SPEED_MEAN,SPEED_SD,EVENT,LON,LAT,Temp,Humidity,Wind,Pressure,prev15,prev30
0,0,1,0,2018-09-01 08:00:00,504.0,99.3,17.19,NO_EVENT,12.291754,41.800588,24.69,0.69,5.04,1014.98,NaN,NaN
1,1,1,0,2018-09-01 08:15:00,481.0,102.1,18.12,NO_EVENT,12.291754,41.800588,24.78,0.69,5.02,1015.01,-23.0,NaN
2,2,1,0,2018-09-01 08:30:00,617.0,99.2,17.91,NO_EVENT,12.291754,41.800588,24.86,0.69,5.07,1015.03,136.0,113.0
3,3,1,0,2018-09-01 08:45:00,562.0,101.7,16.62,NO_EVENT,12.291754,41.800588,24.95,0.69,5.18,1015.06,-55.0,81.0
4,4,1,0,2018-09-01 09:00:00,521.0,99.7,17.06,NO_EVENT,12.291754,41.800588,25.03,0.69,5.35,1015.09,-41.0,-96.0


In [57]:
cal = Italy()

In [58]:
file_final['DATETIME'][0].date()

datetime.date(2018, 9, 1)

In [59]:
feste = []
for i in file_final['DATETIME']:
    feste.append(int(cal.is_working_day(i.date())))

In [60]:
file_final['Working_Days'] = pd.Series(feste, name = 'Working_Days')

In [61]:
file_final.loc[file_final['Working_Days']==0]

,index,ID_SENSOR,DIRECTION,DATETIME,NUM_VEHICLES,SPEED_MEAN,SPEED_SD,EVENT,LON,LAT,Temp,Humidity,Wind,Pressure,prev15,prev30,Working_Days
0,0,1,0,2018-09-01 08:00:00,504.0,99.3,17.19,NO_EVENT,12.291754,41.800588,24.69,0.69,5.04,1014.98,NaN,NaN,0
1,1,1,0,2018-09-01 08:15:00,481.0,102.1,18.12,NO_EVENT,12.291754,41.800588,24.78,0.69,5.02,1015.01,-23.0,NaN,0
2,2,1,0,2018-09-01 08:30:00,617.0,99.2,17.91,NO_EVENT,12.291754,41.800588,24.86,0.69,5.07,1015.03,136.0,113.0,0
3,3,1,0,2018-09-01 08:45:00,562.0,101.7,16.62,NO_EVENT,12.291754,41.800588,24.95,0.69,5.18,1015.06,-55.0,81.0,0
4,4,1,0,2018-09-01 09:00:00,521.0,99.7,17.06,NO_EVENT,12.291754,41.800588,25.03,0.69,5.35,1015.09,-41.0,-96.0,0
5,5,1,0,2018-09-01 09:15:00,448.0,99.2,17.98,NO_EVENT,12.291754,41.800588,25.11,0.67,5.48,1015.09,-73.0,-114.0,0
6,6,1,0,2018-09-01 09:30:00,574.0,97.5,16.57,NO_EVENT,12.291754,41.800588,25.18,0.64,5.65,1015.09,126.0,53.0,0
7,7,1,0,2018-09-01 09:45:00,539.0,97.4,17.95,NO_EVENT,12.291754,41.800588,25.25,0.61,5.87,1015.08,-35.0,91.0,0
8,8,1,0,2018-09-01 10:00:00,525.0,96.7,18.54,NO_EVENT,12.291754,41.800588,25.32,0.59,6.12,1015.08,-14.0,-49.0,0
9,9,1,0,2018-09-01 10:15:00,506.0,97.5,17.60,NO_EVENT,12.291754,41.800588,25.21,0.59,5.81,1015.05,-19.0,-33.0,0


In [62]:
file_final.to_csv('../traffic/file_final.csv')